In [1]:
from cryptography.x509 import load_pem_x509_certificate
import ssl
import socket
%run FunçõesAuxiliares.ipynb 
import os

In [ ]:
pasta_certs = "certificados"
pasta_ca = "AutoridadeCertificadora"
pasta_csrs = "csrs"

os.makedirs(pasta_csrs, exist_ok=True)
os.makedirs(pasta_certs, exist_ok=True)
os.makedirs(pasta_ca, exist_ok=True)

def gerar_chave_e_csr(common_name, country_name, state_or_province_name, locality_name, organization_name):
    key_filename = os.path.join(pasta_certs, f"{common_name}_chave.pem")   # chave salva aqui
    csr_filename = os.path.join(pasta_csrs, f"{common_name}.csr")    # CSR salva aqui

    # Se já existe chave e csr, não gera de novo
    if os.path.exists(key_filename) and os.path.exists(csr_filename):
        print(f"Chave e CSR para {common_name} já existem.")
        return key_filename, csr_filename

    print("Gerando chave privada...")
    key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=4096
    )

    print("Criando CSR...")
    subject = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, country_name),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, state_or_province_name),
        x509.NameAttribute(NameOID.LOCALITY_NAME, locality_name),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, organization_name),
        x509.NameAttribute(NameOID.COMMON_NAME, common_name)
    ])

    csr = x509.CertificateSigningRequestBuilder().subject_name(subject).sign(key, hashes.SHA256(), default_backend())

    with open(key_filename, "wb") as f:
        f.write(key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    with open(csr_filename, "wb") as f:
        f.write(csr.public_bytes(serialization.Encoding.PEM))

    print(f"Chave e CSR criados: {key_filename}, {csr_filename}")
    print(f"Envie o CSR (arquivo '{csr_filename}') para a CA para assinar o certificado.")

    return key_filename, csr_filename


def esperar_certificado(common_name):
    cert_filename = os.path.join(pasta_certs, f"{common_name}_cert.pem")
    print(f"\nAguardando certificado assinado pela CA... ({cert_filename})")

    while not os.path.exists(cert_filename):
        print(f"Certificado ainda não encontrado. Por favor, copie o certificado emitido para este caminho: {cert_filename}")
        resposta = input("Pressione Enter para verificar novamente ou digite 'sair' para cancelar: ").strip().lower()
        if resposta == 'sair':
            print("Cancelando espera pelo certificado.")
            return None  # Retorna None para indicar cancelamento
    print(f"Certificado encontrado: {cert_filename}")
    return cert_filename

def caminho_arquivo(pasta, arquivo):
    return os.path.join(pasta, arquivo)

def chat_continuo_servidor(path_cert_servidor, path_chave_servidor, path_cert_CA, path_cert_dest, chave_aes_servidor, chave_pub_dest, common_name_destinatario):
    context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context.load_cert_chain(certfile=path_cert_servidor, keyfile=path_chave_servidor)
    context.verify_mode = ssl.CERT_NONE 

    if not (validacoes(path_cert_dest, path_cert_CA, common_name_destinatario)):
        print('O Certificado Digital de quem deseja comunicar não passou as devidas validações. Conexão não pode ser iniciada.')
        return

    bindsock = socket.socket()
    bindsock.bind(('0.0.0.0', 4443))
    bindsock.listen(1)

    print("🔐 Servidor TLS do destinatário aguardando conexão segura...")

    conn, addr = bindsock.accept()
    conn_ssl = context.wrap_socket(conn, server_side=True)
    print("✅ Conexão TLS estabelecida com:", addr)

    
    try:
        while True:
            mensagem = conn_ssl.recv(4096)
            if not mensagem:
                print("❌ Cliente desconectou.")
                break

            mensagem = mensagem.decode()
            texto_recebido = desencriptar_mensagem(mensagem, path_chave_servidor)
            print(f"Cliente diz: {texto_recebido}")

            resposta = input("Você: ")
            if resposta.lower() == "sair":
                print("🛑 Encerrando chat.")
                break

            resposta_criptografada = enviar_pacote_json(chave_aes_servidor, resposta, chave_pub_dest)
            print('\n',resposta_criptografada,'\n')

            conn_ssl.sendall(resposta_criptografada)

    finally:
        conn_ssl.close()
        bindsock.close()
        print("🔒 Servidor encerrado.")
        

if __name__ == "__main__":
    print("=== Servidor TLS com chat seguro ===")

    common_name = input("Common Name (CN) do servidor: ")
    common_name_destinatario = input("Common Name (CN) do cliente: ")

    caminho_cert_servidor = caminho_arquivo(pasta_certs, f"{common_name}_cert.pem")
    caminho_chave_servidor = caminho_arquivo(pasta_certs, f"{common_name}_chave.pem")
    caminho_cert_ca = caminho_arquivo(pasta_ca, "certificado_raiz.pem")
    chave_aes_servidor = gerar_chave_aes()
    caminho_cert_dest = caminho_arquivo(pasta_certs, f"{common_name_destinatario}_cert.pem") 

    if not os.path.exists(caminho_cert_servidor):
        # Se não tem certificado, então gera chave e CSR e espera pelo certificado
        print(common_name, " não existe") 
        country_name = input("Country Name (2-letter code): ")
        state_or_province_name = input("State or Province Name: ")
        locality_name = input("Locality Name: ")
        organization_name = input("Organization Name: ")

        caminho_chave_servidor, caminho_csr_cliente = gerar_chave_e_csr(
            common_name, country_name, state_or_province_name, locality_name, organization_name
        )
        caminho_cert_servidor = esperar_certificado(common_name)
        if caminho_cert_servidor is None:
            print("Certificado não foi encontrado. Encerrando.")
            exit(0)
    else:
        print(f"Certificado já existe: {caminho_cert_servidor}")
        
    # Verifica arquivos
    if not os.path.exists(caminho_cert_servidor):
        print(f"Certificado do servidor não encontrado: {caminho_cert_servidor}")
        exit(1)
    if not os.path.exists(caminho_chave_servidor):
        print(f"Chave privada do servidor não encontrada: {caminho_chave_servidor}")
        exit(1)
    if not os.path.exists(caminho_cert_ca):
        print(f"Certificado da CA não encontrado: {caminho_cert_ca}")
        exit(1)
    if not os.path.exists(caminho_cert_dest):
        print(f"Certificado da cliente não encontrado: {caminho_cert_dest}")
        exit(1)
        
    with open(caminho_cert_dest, "rb") as f:
        cert_dest = load_pem_x509_certificate(f.read())
        chave_pub_dest = cert_dest.public_key()

    chat_continuo_servidor(
        caminho_cert_servidor,
        caminho_chave_servidor,
        caminho_cert_ca,
        caminho_cert_dest,
        chave_aes_servidor,
        chave_pub_dest,
        common_name_destinatario
    )



=== Servidor TLS com chat seguro ===


Common Name (CN) do servidor:  Marianna
Common Name (CN) do cliente:  Lourenco


Certificado já existe: certificados\Marianna_cert.pem


C:\Users\maria\OneDrive\Desktop\Faculdade\Criptografia\Grupo 10 D6-Versão Final do Projeto\Grupo 10 D6-Versão Final do Projeto\Código D6 Grupo 10\AutoridadeCertificadora2.py:142: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_before_utc.
  return certificado.not_valid_before <= agora <= certificado.not_valid_after
C:\Users\maria\OneDrive\Desktop\Faculdade\Criptografia\Grupo 10 D6-Versão Final do Projeto\Grupo 10 D6-Versão Final do Projeto\Código D6 Grupo 10\AutoridadeCertificadora2.py:142: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  return certificado.not_valid_before <= agora <= certificado.not_valid_after


🔐 Servidor TLS do destinatário aguardando conexão segura...
